- Update: 2023.04.11

In [1]:
from gevent import monkey
monkey.patch_all()
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions, Chrome
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

# = = = = = = = = = = = = = = = = = =

chromedriver = '../../chromedriver.exe'

# = = = = = = = = = = = = = = = = = =

chrome_options = ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation']) # Chrome正受到自动测试软件的控制
chrome_options.add_argument('--disable-blink-features=AutomationControlled') # window.navigator.webdriver=false
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36')
chrome_options.add_argument('--proxy-server=http://j933.kdltps.com:15818')
chrome_options.add_argument('blink-settings=imagesEnabled=false')
# chrome_options.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./menu-ngk_laser_iridium_spark_plugs.xlsx')
list_product = df_menu['Product'].to_list()
print('总数量：' + str(len(list_product)))
print()

# = = = = = = = = = = = = = = = = = =

df = pd.DataFrame(columns=['Item',
                           '标题',
                           '图片',
                           'Brand',
                           'Part Number',
                           'UPC',
                           '车型',
                           'Center Electrode Material',
                           'Ground Electrode Material',
                           'Electrode Material',
                           '链接',
                           '图片链接'])

# = = = = = = = = = = = = = = = = = =

work = Queue()
for product in list_product:
    work.put_nowait(product)

# = = = = = = = = = = = = = = = = = =

def crawler():
    global df
    
    while not work.empty():
        product = work.get_nowait()
        url = product
        driver = Chrome(service=Service(chromedriver), options=chrome_options)
        driver.maximize_window()
        time.sleep(0.3)
        driver.maximize_window()
        
        for i in range(11):
            try:
                driver.get(url)
                driver.maximize_window()

                # = = = = = = = = = = = = = = = = = =

                WebDriverWait(driver, 30, 0.5, None).until(EC.presence_of_element_located((By.CLASS_NAME, 'prod-title.js-prod-title')))
                title = driver.find_element(By.CLASS_NAME, 'prod-title.js-prod-title').find_element(By.TAG_NAME, 'h1').text
                item = driver.find_element(By.CLASS_NAME, 'prod-title.js-prod-title').find_element(By.TAG_NAME, 'div').text.split('|')[1].replace('Item # ', '')
                pic = driver.find_element(By.XPATH, '//*[@class="product-image-main"]').get_attribute('href')

                # = = = = = = = = = = = = = = = = = =

                try:
                    specifications = driver.find_element(By.XPATH, '//*[@class="spec-stripped-table"]').text

                    center_electrode_material = re.findall('\nCenter Electrode\nMaterial: .*?\n', specifications)[0].replace('Center Electrode\nMaterial: ', '').replace('\n', '')
                    ground_electrode_material = re.findall('\nGround Electrode\nMaterial: .*?\n', specifications)[0].replace('Ground Electrode\nMaterial: ', '').replace('\n', '')
                    electrode_material = re.findall('\nElectrode Material: .*?\n', specifications)[0].replace('Electrode Material: ', '').replace('\n', '')
                except:
                    center_electrode_material = ''
                    ground_electrode_material = ''
                    electrode_material = ''
                
                # = = = = = = = = = = = = = = = = = =

                brand = ''
                part_number = ''
                upc = ''
                WebDriverWait(driver, 30, 0.5, None).until(EC.presence_of_element_located((By.XPATH, '//*[@class="prod-offer-item"]')))
                list_feature = driver.find_elements(By.XPATH, '//*[@class="prod-offer-item"]')
                for feature in list_feature:
                    if feature.text.startswith('Brand'):
                        brand = feature.text.split(':')[1]
                    elif feature.text.startswith('Part Number'):
                        part_number = feature.text.split(':')[1]
                    elif feature.text.startswith('UPC'):
                        upc = feature.text.split(':')[1]

                # = = = = = = = = = = = = = = = = = =

                for j in driver.find_elements(By.XPATH, '//*[@class="mmy-row-expand"]')[1:]:
                    j.click()
                    time.sleep(0.5)
                    
                # = = = = = = = = = = = = = = = = = =

                list_make = [j.text for j in driver.find_elements(By.XPATH, '//*[@class="mmy-row-title pointer js-mmy-row-expand"]')]
                list_model = [j.text for j in driver.find_elements(By.XPATH, '//*[@class="mmy-row-description"]')]

                list_vehicle = []
                for j in range(len(list_make)):
                    list_vehicle.extend([list_make[j] + ' ' + k for k in list_model[j].split('\n')])

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'Item': [item],
                                        '标题': [title],
                                        '图片': [''],
                                        'Brand': [brand],
                                        'Part Number': [part_number],
                                        'UPC': [upc],
                                        '车型': [vehicle],
                                        'Center Electrode Material': [center_electrode_material],
                                        'Ground Electrode Material': [ground_electrode_material],
                                        'Electrode Material': [electrode_material],
                                        '链接': [url],
                                        '图片链接': [pic]})
                
                break

            except:
                df_temp = pd.DataFrame({'Item': [''],
                                        '标题': [''],
                                        '图片': [''],
                                        'Brand': [''],
                                        'Part Number': [''],
                                        'UPC': [''],
                                        '车型': [''],
                                        'Center Electrode Material': [''],
                                        'Ground Electrode Material': [''],
                                        'Electrode Material': [''],
                                        '链接': [url],
                                        '图片链接': ['']})
                continue

        # = = = = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')
        
        print(product, ' [' + str(i) + '] - 剩余数量：' + str(work.qsize()))
        driver.close()
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df.to_excel('./crawler.xlsx', index=False)
print()
print('搞定')

batch：0 - 总数量：15

https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-95927.html  [0] - 剩余数量：7
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-91731.html?singleid=5087660506  [0] - 剩余数量：6
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-93560.html?singleid=5087720497  [0] - 剩余数量：5
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-92154.html?singleid=5087784345  [0] - 剩余数量：4
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-90911.html?singleid=5087850343  [0] - 剩余数量：3
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-96206.html?singleid=5087797357  [0] - 剩余数量：2
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-96698.html?singleid=5087889357  [0] - 剩余数量：1
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-95927.html?singleid=5087752498  [0] - 剩余数量：0
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-94524.html?singleid=5087752490  [1] - 剩余数量：0
https://www.carid.com/ngk/laser-iridium-spark-plug-mpn-95927.html?singleid=5087757502  [0] - 剩余数量：0
ht

D:\Lennon\python\venv\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
